In [9]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import cv2 
import numpy as np

In [10]:
model = YOLO("yolov9e")

In [11]:
parking_slots = [
    [(1603, 834), (1785,885),(1754, 1047), (1560, 1040)],#1
    [(1468, 745), (1615, 799),(1555, 1037), (1401, 928)],#2
    [(1355, 677), (1471,710),(1393, 919), (1268, 832)],#3
    [(1250, 622), (1342,662),(1265, 826), (1164, 752)],#4
    [(1176, 571), (1257,603),(1160, 748), (1077, 683)],#5
    [(1101, 528), (1151,570),(1075, 680), (1001, 627)],#6
    [(1031, 486), (1102,516),(1000, 626), (937, 581)],#7
    [(976, 439), (1037,482),(925, 581), (876, 538)],#8
    [(935, 411), (992,432),(876, 536), (831, 502)],#9
    [(881, 375), (941,418),(830, 494), (786, 471)]#10
] 


In [12]:

# the function to write the detected slots in a txt file
def write_in_txt(filename, detected_occupied_slots):
    with open(filename, 'w') as file:
        file.write(f"{len(detected_occupied_slots)}\n")
        for slot, status in detected_occupied_slots.items():
            file.write(f"{slot} {status}\n")
            
# here we check if the center of the car is in the slot
def is_center_in_slot(center, slot):
    slot_arr = np.array(slot, np.int32).reshape((-1, 1, 2))
    return cv2.pointPolygonTest(slot_arr, center, False) >= 0

def predict_detections(image_path, query_path, parking_slots):
    
    # we load the image and convert it to RGB
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # perform the inference with classes car and trucks (2, 7)
    results = model.predict(image_rgb, classes=[2, 7], verbose=False)
    
    # get the bounding boxes and classes
    detections = results[0].boxes.xyxy  #bounding boxes
    classes = results[0].boxes.cls  #classes
     
    #here we calculate the center of detections
    centers = []
    for detection, cls in zip(detections, classes):
        if int(cls) == 2 or int(cls) == 7:  # Class id 2 corresponds to 'car' in COCO dataset
            x_min, y_min, x_max, y_max = detection[:4] #here we get the coordinates of the bounding box
            center_x = int((x_min + x_max) / 2) #we calculate the average of the x coordinates
            center_y = int((y_min + y_max) / 2) #calculate the average of the y coordinates
            centers.append((center_x, center_y)) #for every center calculated we append it to the centers list
    
    #load the query file to get the slots to check
    with open(query_path, 'r') as file:
        lines = file.readlines()
        slots_to_check = [int(line.strip()) for line in lines[1:]] 
    
    #we create a dictionary to store the detected occupied slots
    detected_occupied_slots = {slot: 0 for slot in slots_to_check} #initialize all slots as empty
    for (center_x, center_y) in centers: 
        point_center = (center_x, center_y) #we create a tuple with the center of the car
        for index, slot in enumerate(parking_slots): 
            if (index + 1) in slots_to_check and is_center_in_slot(point_center, slot): #if the center of the car is in the slot
                detected_occupied_slots[index + 1] = 1 #we mark the slot as occupied
    
    return detected_occupied_slots

In [14]:
import os
from tqdm import tqdm

#paths for folders
image_folder = r'D:\Radu\fac\master\sem2\ComputerVision\Project2\train\Task1'
query_folder = r'D:\Radu\fac\master\sem2\ComputerVision\Project2\train\Task1'
output_folder = r'D:\Radu\fac\master\sem2\ComputerVision\Project2\train\Task1\output'

#create the output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)

# get all image files from the image folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

for filename in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(image_folder, filename)
    query_path = os.path.join(query_folder, filename.replace('.jpg', '_query.txt'))
    output_path = os.path.join(output_folder, filename.replace('.jpg', '_det.txt'))
    
    detected_occupied_slots = predict_detections(image_path, query_path, parking_slots)
      
    write_in_txt(output_path, detected_occupied_slots)

print("Detection completed.")

Processing images:  10%|█         | 5/50 [00:22<03:26,  4.59s/it]


KeyboardInterrupt: 